In [ ]:
import os
import spacy
nlp = spacy.load("en_core_web_sm")
from google.colab import drive
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import itertools
import random
import nltk
import json
import requests
import urllib.request
from bs4 import BeautifulSoup
import tarfile
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import regex as re
import glob
from scipy.special import expit
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Annotations Model

In [ ]:
drive.mount("drive")
baselinedf = pd.read_table('drive/MyDrive/Coding Portfolio/NewsScraperProject/Datasets/wikisent2.txt', header=None)
baselinedf = baselinedf[1000000:6000000]
cutbaselinedf = pd.DataFrame(np.empty((4000, 2)))
cutbaselinedf.columns = ["label", "text"]
sentencelist1 = []
cutbaselinedf = cutbaselinedf.assign(label='Non-biased')
for i in range(4000):
  sentencelist1.append(baselinedf[0][random.randint(1000000, 6000000)])
cutbaselinedf["text"] = sentencelist1
display(cutbaselinedf)

Mounted at drive


,label,text
0,Non-biased,"National Product is a rock band from Oahu, Haw..."
1,Non-biased,In the 1990s he managed the Black Cat night cl...
2,Non-biased,"In the past, for instance, farmers have used t..."
3,Non-biased,"The 33rd Golden Raspberry Awards, or Razzies, ..."
4,Non-biased,She succeeded Rachel Florence Appoh who had re...
...,...,...
3995,Non-biased,Coronel Hilario Lagos is a village and rural l...
3996,Non-biased,"Depending on the type of the sign, it might al..."
3997,Non-biased,The cover is from the silent movie Les Vampire...
3998,Non-biased,EverCharge offers wireless networked charging ...


In [ ]:
drive.mount("drive")
biasdf2 = pd.read_excel('drive/MyDrive/Coding Portfolio/NewsScraperProject/Datasets/annotations.xlsx', header=None)
#columns 8, 12, 13, and 14 are useful but are not used for code in line 8
biasdf2.drop(columns=[0, 1, 2, 3, 4, 5, 7, 9, 11, 15, 16, 17, 18, 19 , 20, 21, 22, 23, 8, 12, 13, 14 ], inplace=True)
biasdf2.rename(columns={6:"label", 10:"text"}, inplace=True)
biasdf2.dropna(inplace=True)
print(biasdf2["label"].value_counts())
# concat with nonbias dataframe made above
frames = [cutbaselinedf, biasdf2]
biasdf3 = pd.concat(frames)
display(biasdf2)
def numerateBias(label):
  if label == "Non-biased":
    return 0
  else:
    return 1
  return 1
biasdf3["label"] = biasdf3["label"].apply(numerateBias)


Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
Biased        10651
Non-biased     7124
label             1
Name: label, dtype: int64


,label,text
0,label,text
1,Non-biased,The transgender effort to suppress any recogni...
2,Non-biased,Radical Virginia Citizens Defense League has o...
3,Non-biased,Miller is the architect of President Donald Tr...
4,Non-biased,"The House Democrats’ 1,400-page coronavirus re..."
...,...,...
17771,Biased,Oregon first graders could attend gun safety c...
17772,Biased,But President Donald Trump and his campaign ad...
17773,Biased,The normalization effort included softened rhe...
17774,Biased,"And so, while demonstrations began to spread a..."


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(biasdf3["text"], biasdf3["label"], test_size=0.2, random_state=42)

pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', LogisticRegression())])
#--------Fitting the model--------
biasmodel = pipe.fit(X_train, y_train)
#Accuracy
prediction = biasmodel.predict(X_test)

print("accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))
print((mean_absolute_error(y_test, prediction)))
print(f1_score(y_test, prediction))

accuracy: 73.81%
0.26193755739210284
0.7431915372496062



# Web Scraping



In [ ]:
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
def splitSenNltk(text):
    return nltk.sent_tokenize(text) # this gives us a list of sentences

def splitSen(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences
#Saerch for P Tag
def scrapeBody(url):
  URL_raw = url
  URL = URL_raw.strip().replace(" ", "+")
  try:
    page = requests.get(URL)
  except:
    print("Could not find URL..")
  htmlParse = BeautifulSoup(page.content, 'html.parser')
  input = ""
  for para in htmlParse.find_all("p"):
      input = input + (para.get_text()) + " "
  sentenceList = splitSenNltk(input)
  return (sentenceList)

#Searching for A tag as hyperlinks for more links
def scrapeHyper(url):
  URL_raw = url
  URL = URL_raw.strip().replace(" ", "+")
  try:
    page = requests.get(URL)
  except:
    print("Could not find URL..")
  htmlParse = BeautifulSoup(page.content, 'html5lib')
  linkList = []
  for link in htmlParse.find_all('a', href=True):
      linkList.append(link['href'])
  return (linkList)

#h1 - h6 (scraping Headlines)
def scrapeHead(url):
  URL_raw = url
  URL = URL_raw.strip().replace(" ", "+")
  try:
    page = requests.get(URL)
  except:
    print("Could not find URL..")
  htmlParse = BeautifulSoup(page.content, 'html.parser')
  sentenceList = []
  for para in htmlParse.find_all("span"):
      sentenceList.append(para.get_text())
  return (sentenceList)

In [ ]:
dfMain1 = pd.DataFrame(columns = ['sentence', 'ideology', 'publisher', 'link'])

In [ ]:
#huffpost
counter = 0
linkList = []
dfHuff = pd.DataFrame(columns = ['sentence', 'ideology', 'publisher', 'link'])
for j in range(1,2):
  for i in (scrapeHyper('https://www.huffpost.com/news/politics?page=' + str(j))):
    counter+=1
    if('/entry' in i and counter%2==0):
      linkList.append(i)

for i in linkList:
  dfTemp = pd.DataFrame(columns = ['sentence', 'ideology', 'publisher', 'link'])
  dfTemp['sentence'] = scrapeBody(i)
  dfTemp['link'] = i
  dfHuff = dfHuff.append(dfTemp, ignore_index=True)

dfHuff['ideology'] = 0.5
dfHuff['publisher'] = 'huffpost'
display(dfHuff)

,sentence,ideology,publisher,link
0,"Senior National Correspondent, HuffPost This a...",0.5,huffpost,https://www.huffpost.com/entry/gun-violence-me...
1,Click here to subscribe.,0.5,huffpost,https://www.huffpost.com/entry/gun-violence-me...
2,Outbursts of gun violence always get Republica...,0.5,huffpost,https://www.huffpost.com/entry/gun-violence-me...
3,It would be a lot easier to take that talk ser...,0.5,huffpost,https://www.huffpost.com/entry/gun-violence-me...
4,Texas Gov.,0.5,huffpost,https://www.huffpost.com/entry/gun-violence-me...
...,...,...,...,...
403,The Kremlin said Putin held an 80-minute telep...,0.5,huffpost,https://www.huffpost.com/entry/us-ukraine-war-...
404,"Overall, the United States has committed appro...",0.5,huffpost,https://www.huffpost.com/entry/us-ukraine-war-...
405,___ AP Diplomatic Writer Matthew Lee and Assoc...,0.5,huffpost,https://www.huffpost.com/entry/us-ukraine-war-...
406,Do you have information you want to share with...,0.5,huffpost,https://www.huffpost.com/entry/us-ukraine-war-...


In [ ]:
#BrietBart
counter = 0
linkList = []
dfFox = pd.DataFrame(columns = ['sentence', 'ideology', 'publisher', 'link'])
for j in range(1,2):
  for i in (scrapeHyper('https://www.huffpost.com/news/politics?page=' + str(j))):
    counter+=1
    if('/entry' in i and counter%2==0):
      linkList.append(i)

for i in linkList:
  dfTemp = pd.DataFrame(columns = ['sentence', 'ideology', 'publisher', 'link'])
  dfTemp['sentence'] = scrapeBody(i)
  dfTemp['link'] = i
  dfHuff = dfHuff.append(dfTemp, ignore_index=True)

dfHuff['ideology'] = 0.5
dfHuff['publisher'] = 'huffpost'
display(dfHuff)

,sentence,ideology,publisher,link
0,"Senior National Correspondent, HuffPost This a...",0.5,huffpost,https://www.huffpost.com/entry/gun-violence-me...
1,Click here to subscribe.,0.5,huffpost,https://www.huffpost.com/entry/gun-violence-me...
2,Outbursts of gun violence always get Republica...,0.5,huffpost,https://www.huffpost.com/entry/gun-violence-me...
3,It would be a lot easier to take that talk ser...,0.5,huffpost,https://www.huffpost.com/entry/gun-violence-me...
4,Texas Gov.,0.5,huffpost,https://www.huffpost.com/entry/gun-violence-me...
...,...,...,...,...
811,The Kremlin said Putin held an 80-minute telep...,0.5,huffpost,https://www.huffpost.com/entry/us-ukraine-war-...
812,"Overall, the United States has committed appro...",0.5,huffpost,https://www.huffpost.com/entry/us-ukraine-war-...
813,___ AP Diplomatic Writer Matthew Lee and Assoc...,0.5,huffpost,https://www.huffpost.com/entry/us-ukraine-war-...
814,Do you have information you want to share with...,0.5,huffpost,https://www.huffpost.com/entry/us-ukraine-war-...


In [ ]:
scrapeHyper('https://web.archive.org/web/*/https://www.foxnews.com/politics*')

['https://archive.org']

In [ ]:
for i in scrapeBody('https://www.foxnews.com/politics/manchin-biden-title-42-border-policy-frightening-decision'):
  print(i)
print("===============================")
for i in scrapeBody('https://occupydemocrats.com/2020/04/06/wisconsins-gop-stacked-supreme-court-forces-state-to-hold-primary-during-pandemic/'):
  print(i)


      This material may not be published, broadcast, rewritten,
      or redistributed.
©2022 FOX News Network, LLC.
All rights reserved.
Quotes displayed in real-time or delayed by at least 15 minutes.
Market data provided by Factset.
Powered and implemented by FactSet Digital Solutions.
Legal Statement.
Mutual Fund and ETF data provided by Refinitiv Lipper.
Texas Lt. Gov.
Dan Patrick weighs in as the Biden administration considers ending Title 42, which allows for the rapid expulsion of migrants.
Sen. Joe Manchin, D-W.Va., on Friday tore into the Biden administration’s move to lift the Title 42 border policy at the end of May, calling it a "frightening decision" and warning that the extraordinary numbers will "only get worse" in the months ahead.
"Today’s announcement by the [Centers for Disease Control] and the Biden administration is a frightening decision," Manchin said in a statement.
BIDEN ADMINISTRATION TO LIFT TITLE 42 AT END OF MAY, DESPITE FEARS OF LOOMING MIGRANT WAVE The 